In [1]:
from google.colab import drive
drive.mount('/content/drive')

%tensorflow_version 1.x
import tensorflow as tf

import numpy as np
from PIL import Image
import os
import time
import datetime
from sklearn import metrics

LABELS = ["PASCOLO", "BOSCO", "NON COLTIVABILE"]

N_of_classes = len(LABELS) # Number of classes

batch_size = 32

patch_size = 64

half_ps = int(patch_size/2)

quarter_ps = int(patch_size/4)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
TensorFlow 1.x selected.


In [2]:
stride_value = 3 # segnet_value = 3

conv_stride = 1 #stride of convolutional layer during image compression
conv_stride_d = 1 #stride of convolutional layer during image decompression
deconv_stride = 2 #stride of deconvolutional layer during image compression
pool_stride = 2 #stride of max pooling layer apllied during compression

pool_kernel = 2 #kernel dim for deconv and maxpool
conv_kernel = 3 #kernel dim for convolutional layer during image compression
conv_kernel_d = 1 #kernel dim for convolutional layer during image decompression

N_of_band = 4 #image band 3 for RGB + 1 for NIR


###############################################################################
####################### ~ NETWORK IMPLEMENTATION ~ ############################

def maxpool2d(x, _name, hw_kernel_dim = pool_kernel, nc_kernel_dim = 1, nc_strides = 1, hw_strides = pool_stride):
  return tf.nn.max_pool(x,
  ksize = [nc_kernel_dim, hw_kernel_dim, hw_kernel_dim, nc_kernel_dim],
  strides = [nc_strides, hw_strides, hw_strides, nc_strides],
  padding = 'SAME',
  data_format = 'NHWC',
  name = _name)

def conv2d(x, W, b, _name, hw_strides, nc_strides = 1):
  # Conv2D wrapper, with bias and relu activation
  # W contains the filter shape
  '''
  La conv2d prende in input il dato entrante x poi W rappresenta il filtro, che sono poi i pesi
  questo deve avere la forma corretta: [filter_height, filter_width, in_channels, out_channels]
  La stride, non mi è chiarissimo perche ma deve avere sempre 1 1, poi rispetto a pytorch anche gli
  altri due valori devono essere uguali
  '''
  x = tf.nn.conv2d(x,
  W,
  strides=[nc_strides, hw_strides, hw_strides, nc_strides],
  padding='SAME',
  data_format='NHWC',
  name = _name)

  x = tf.nn.bias_add(x, b, data_format='NHWC')
  return tf.nn.relu(x) 

def deconv2d(x, n_of_filters, _kernel, _name, strides = deconv_stride):
  x = tf.layers.conv2d_transpose(x,
  n_of_filters,
  _kernel,
  strides=(strides, strides),
  padding = 'SAME',
  data_format='channels_last',
  name = _name)
  return x


def net(x, weights, biases):
  '''La conv2d prende in input '''
  x = conv2d(x, weights['wc1_1'], biases['bc1_1'], _name ='conv_1_1', hw_strides = conv_stride)
  x = tf.layers.batch_normalization(x, name='batch_norm_1_1')
  x = conv2d(x, weights['wc1_2'], biases['bc1_2'], _name ='conv_1_2', hw_strides = conv_stride)
  x = tf.layers.batch_normalization(x, name='batch_norm_1_2')
  x = maxpool2d(x, _name ='maxpool_1')
  print(x)

  x = conv2d(x, weights['wc2_1'], biases['bc2_1'], _name ='conv_2_1', hw_strides = conv_stride)
  x = tf.layers.batch_normalization(x, name='batch_norm_2_1')
  x = conv2d(x, weights['wc2_2'], biases['bc2_2'], _name ='conv_2_2', hw_strides = conv_stride)
  x = tf.layers.batch_normalization(x, name='batch_norm_221')
  x = maxpool2d(x, _name ='maxpool2')
  print(x)
  x = conv2d(x, weights['wc3_1'], biases['bc3_1'], _name ='conv_3_1', hw_strides = conv_stride)
  x = tf.layers.batch_normalization(x, name='batch_norm_3_1')
  x = conv2d(x, weights['wc3_2'], biases['bc3_2'], _name ='conv_3_2', hw_strides = conv_stride)
  x = tf.layers.batch_normalization(x, name='batch_norm_3_2')
  x = conv2d(x, weights['wc3_3'], biases['bc3_3'], _name ='conv_3_3', hw_strides = conv_stride)
  x = tf.layers.batch_normalization(x, name='batch_norm_3_3')
  x = maxpool2d(x, _name ='maxpool3')
  print(x)
  x = conv2d(x, weights['wc4_1'], biases['bc4_1'], _name ='conv_4_1', hw_strides = conv_stride)
  x = tf.layers.batch_normalization(x, name='batch_norm_4_1')
  x = conv2d(x, weights['wc4_2'], biases['bc4_2'], _name ='conv_4_2', hw_strides = conv_stride)
  x = tf.layers.batch_normalization(x, name='batch_norm_4_2')
  x = conv2d(x, weights['wc4_3'], biases['bc4_3'], _name ='conv_4_3', hw_strides = conv_stride)
  x = tf.layers.batch_normalization(x, name='batch_norm_4_3')
  x = maxpool2d(x, _name ='maxpool4')
  print(x)
  x = conv2d(x, weights['wc5_1'], biases['bc5_1'], _name ='conv_5_1', hw_strides = conv_stride)
  x = tf.layers.batch_normalization(x, name='batch_norm_5_1')
  x = conv2d(x, weights['wc5_2'], biases['bc5_2'], _name ='conv_5_2', hw_strides = conv_stride)
  x = tf.layers.batch_normalization(x, name='batch_norm_5_2')
  x = conv2d(x, weights['wc5_3'], biases['bc5_3'], _name ='conv_5_3', hw_strides = conv_stride)
  x = tf.layers.batch_normalization(x, name='batch_norm_5_3')
  x = maxpool2d(x, _name ='maxpool5') 
  print(x)
  x = deconv2d(x, 512, conv_kernel, _name ='deconv1') 
  x = conv2d(x, weights['wc_d_5_1'], biases['bc_d_5_1'], _name ='conv_d_5_1', hw_strides = conv_stride_d)
  x = tf.layers.batch_normalization(x, name='batch_norm_d_5_1')
  x = conv2d(x, weights['wc_d_5_2'], biases['bc_d_5_2'], _name ='conv_d_5_2', hw_strides = conv_stride_d)
  x = tf.layers.batch_normalization(x, name='batch_norm_d_5_2')
  x = conv2d(x, weights['wc_d_5_3'], biases['bc_d_5_3'], _name ='conv_d_5_3', hw_strides = conv_stride_d)
  x = tf.layers.batch_normalization(x, name='batch_norm_d_5_3')
  print(x)
  x = deconv2d(x, 512, pool_kernel, _name ='deconv2')
  x = conv2d(x, weights['wc_d_4_1'], biases['bc_d_4_1'], _name ='conv_d_4_1', hw_strides = conv_stride_d)
  x = tf.layers.batch_normalization(x, name='batch_norm_d_4_1')
  x = conv2d(x, weights['wc_d_4_2'], biases['bc_d_4_2'], _name ='conv_d_4_2', hw_strides = conv_stride_d)
  x = tf.layers.batch_normalization(x, name='batch_norm_d_4_2')
  x = conv2d(x, weights['wc_d_4_3'], biases['bc_d_4_3'], _name ='conv_d_4_3', hw_strides = conv_stride_d)
  x = tf.layers.batch_normalization(x, name='batch_norm_d_4_3')
  print(x)
  x = deconv2d(x, 256, pool_kernel, _name ='deconv3')
  x = conv2d(x, weights['wc_d_3_1'], biases['bc_d_3_1'], _name ='conv_d_3_1', hw_strides = conv_stride_d)
  x = tf.layers.batch_normalization(x, name='batch_norm_d_3_1')
  x = conv2d(x, weights['wc_d_3_2'], biases['bc_d_3_2'], _name ='conv_d_3_2', hw_strides = conv_stride_d)
  x = tf.layers.batch_normalization(x, name='batch_norm_d_3_2')
  x = conv2d(x, weights['wc_d_3_3'], biases['bc_d_3_3'], _name ='conv_d_3_3', hw_strides = conv_stride_d)
  x = tf.layers.batch_normalization(x, name='batch_norm_d_3_3')
  print(x)
  x = deconv2d(x, 128, pool_kernel, _name ='deconv4')
  x = conv2d(x, weights['wc_d_2_1'], biases['bc_d_2_1'], _name ='conv_d_2_1', hw_strides = conv_stride_d)
  x = tf.layers.batch_normalization(x, name='batch_norm_d_2_1')
  x = conv2d(x, weights['wc_d_2_2'], biases['bc_d_2_2'], _name ='conv_d_2_2', hw_strides = conv_stride_d)
  x = tf.layers.batch_normalization(x, name='batch_norm_d_2_2')
  print(x)
  x = deconv2d(x, 64, pool_kernel, _name ='deconv5')
  x = conv2d(x, weights['wc_d_1_1'], biases['bc_d_1_1'], _name ='conv_d_1_1', hw_strides = conv_stride_d)
  x = tf.layers.batch_normalization(x, name='batch_norm_d_1_1')
  x = conv2d(x, weights['wc_d_1_2'], biases['bc_d_1_2'], _name ='conv_d_1_2', hw_strides = conv_stride_d)

  x = tf.nn.softmax(x, axis = 3,name ='softmax')
  print(x)
  return x



biases = {
'bc1_1': tf.get_variable('B0', shape=(64), initializer=tf.contrib.layers.xavier_initializer()), 
'bc1_2': tf.get_variable('B1', shape=(64), initializer=tf.contrib.layers.xavier_initializer()), 
'bc2_1': tf.get_variable('B2', shape=(128), initializer=tf.contrib.layers.xavier_initializer()), 
'bc2_2': tf.get_variable('B3', shape=(128), initializer=tf.contrib.layers.xavier_initializer()), 
'bc3_1': tf.get_variable('B4', shape=(256), initializer=tf.contrib.layers.xavier_initializer()),
'bc3_2': tf.get_variable('B5', shape=(256), initializer=tf.contrib.layers.xavier_initializer()), 
'bc3_3': tf.get_variable('B6', shape=(256), initializer=tf.contrib.layers.xavier_initializer()), 
'bc4_1': tf.get_variable('B7', shape=(512), initializer=tf.contrib.layers.xavier_initializer()), 
'bc4_2': tf.get_variable('B8', shape=(512), initializer=tf.contrib.layers.xavier_initializer()), 
'bc4_3': tf.get_variable('B9', shape=(512), initializer=tf.contrib.layers.xavier_initializer()), 
'bc5_1': tf.get_variable('B10', shape=(512), initializer=tf.contrib.layers.xavier_initializer()), 
'bc5_2': tf.get_variable('B11', shape=(512), initializer=tf.contrib.layers.xavier_initializer()), 
'bc5_3': tf.get_variable('B12', shape=(512), initializer=tf.contrib.layers.xavier_initializer()), 
'bc_d_5_1': tf.get_variable('B13', shape=(512), initializer=tf.contrib.layers.xavier_initializer()), 
'bc_d_5_2': tf.get_variable('B14', shape=(512), initializer=tf.contrib.layers.xavier_initializer()), 
'bc_d_5_3': tf.get_variable('B15', shape=(512), initializer=tf.contrib.layers.xavier_initializer()), 
'bc_d_4_1': tf.get_variable('B16', shape=(512), initializer=tf.contrib.layers.xavier_initializer()), 
'bc_d_4_2': tf.get_variable('B17', shape=(512), initializer=tf.contrib.layers.xavier_initializer()), 
'bc_d_4_3': tf.get_variable('B18', shape=(256), initializer=tf.contrib.layers.xavier_initializer()), 
'bc_d_3_1': tf.get_variable('B19', shape=(256), initializer=tf.contrib.layers.xavier_initializer()), 
'bc_d_3_2': tf.get_variable('B20', shape=(256), initializer=tf.contrib.layers.xavier_initializer()), 
'bc_d_3_3': tf.get_variable('B21', shape=(128), initializer=tf.contrib.layers.xavier_initializer()), 
'bc_d_2_1': tf.get_variable('B22', shape=(128), initializer=tf.contrib.layers.xavier_initializer()), 
'bc_d_2_2': tf.get_variable('B23', shape=(64), initializer=tf.contrib.layers.xavier_initializer()), 
'bc_d_1_1': tf.get_variable('B24', shape=(64), initializer=tf.contrib.layers.xavier_initializer()), 
'bc_d_1_2': tf.get_variable('B25', shape=(N_of_classes), initializer=tf.contrib.layers.xavier_initializer()),
}

weights = {
'wc1_1': tf.get_variable('W0', shape=(conv_kernel,conv_kernel,4,64), initializer=tf.contrib.layers.xavier_initializer()), 
'wc1_2': tf.get_variable('W1', shape=(conv_kernel,conv_kernel,64,64), initializer=tf.contrib.layers.xavier_initializer()), 
'wc2_1': tf.get_variable('W2', shape=(conv_kernel,conv_kernel,64,128), initializer=tf.contrib.layers.xavier_initializer()), 
'wc2_2': tf.get_variable('W3', shape=(conv_kernel,conv_kernel,128,128), initializer=tf.contrib.layers.xavier_initializer()), 
'wc3_1': tf.get_variable('W4', shape=(conv_kernel,conv_kernel,128,256), initializer=tf.contrib.layers.xavier_initializer()),
'wc3_2': tf.get_variable('W5', shape=(conv_kernel,conv_kernel,256,256), initializer=tf.contrib.layers.xavier_initializer()), 
'wc3_3': tf.get_variable('W6', shape=(conv_kernel,conv_kernel,256,256), initializer=tf.contrib.layers.xavier_initializer()), 
'wc4_1': tf.get_variable('W7', shape=(conv_kernel,conv_kernel,256,512), initializer=tf.contrib.layers.xavier_initializer()), 
'wc4_2': tf.get_variable('W8', shape=(conv_kernel,conv_kernel,512,512), initializer=tf.contrib.layers.xavier_initializer()), 
'wc4_3': tf.get_variable('W9', shape=(conv_kernel,conv_kernel,512,512), initializer=tf.contrib.layers.xavier_initializer()), 
'wc5_1': tf.get_variable('W10', shape=(conv_kernel,conv_kernel,512,512), initializer=tf.contrib.layers.xavier_initializer()), 
'wc5_2': tf.get_variable('W11', shape=(conv_kernel,conv_kernel,512,512), initializer=tf.contrib.layers.xavier_initializer()), 
'wc5_3': tf.get_variable('W12', shape=(conv_kernel,conv_kernel,512,512), initializer=tf.contrib.layers.xavier_initializer()), 
'wc_d_5_1': tf.get_variable('W13', shape=(conv_kernel_d,conv_kernel_d,512,512), initializer=tf.contrib.layers.xavier_initializer()), 
'wc_d_5_2': tf.get_variable('W14', shape=(conv_kernel_d,conv_kernel_d,512,512), initializer=tf.contrib.layers.xavier_initializer()), 
'wc_d_5_3': tf.get_variable('W15', shape=(conv_kernel_d,conv_kernel_d,512,512), initializer=tf.contrib.layers.xavier_initializer()), 
'wc_d_4_1': tf.get_variable('W16', shape=(conv_kernel_d,conv_kernel_d,512,512), initializer=tf.contrib.layers.xavier_initializer()), 
'wc_d_4_2': tf.get_variable('W17', shape=(conv_kernel_d,conv_kernel_d,512,512), initializer=tf.contrib.layers.xavier_initializer()), 
'wc_d_4_3': tf.get_variable('W18', shape=(conv_kernel_d,conv_kernel_d,512,256), initializer=tf.contrib.layers.xavier_initializer()), 
'wc_d_3_1': tf.get_variable('W19', shape=(conv_kernel_d,conv_kernel_d,256,256), initializer=tf.contrib.layers.xavier_initializer()), 
'wc_d_3_2': tf.get_variable('W20', shape=(conv_kernel_d,conv_kernel_d,256,256), initializer=tf.contrib.layers.xavier_initializer()), 
'wc_d_3_3': tf.get_variable('W21', shape=(conv_kernel_d,conv_kernel_d,256,128), initializer=tf.contrib.layers.xavier_initializer()), 
'wc_d_2_1': tf.get_variable('W22', shape=(conv_kernel_d,conv_kernel_d,128,128), initializer=tf.contrib.layers.xavier_initializer()), 
'wc_d_2_2': tf.get_variable('W23', shape=(conv_kernel_d,conv_kernel_d,128,64), initializer=tf.contrib.layers.xavier_initializer()), 
'wc_d_1_1': tf.get_variable('W24', shape=(conv_kernel_d,conv_kernel_d,64,64), initializer=tf.contrib.layers.xavier_initializer()), 
'wc_d_1_2': tf.get_variable('W25', shape=(conv_kernel_d,conv_kernel_d,64,N_of_classes), initializer=tf.contrib.layers.xavier_initializer()),
}

############################# ~ TF OPERATIONS ~ ###############################
len_x = 3000
len_y = 3400
#Network placeholder
x = tf.placeholder("float32", shape=[1, patch_size, patch_size, N_of_band])

pred_patch = tf.placeholder("float32", shape=[1, patch_size, patch_size, N_of_classes])
y = tf.placeholder("float32", shape=[1, patch_size, patch_size, N_of_classes])

mappa = tf.placeholder("float32", shape=[1,len_x, len_y, N_of_classes])
gt = tf.placeholder("float32", shape=[1,len_x, len_y, N_of_classes])
#Network prediction
predictions = net(x,weights,biases)

#Accuracuy
accuracy_tot = tf.count_nonzero(tf.keras.metrics.categorical_accuracy(mappa,gt))
accuracy_patch = tf.count_nonzero(tf.keras.metrics.categorical_accuracy(pred_patch,y))

norm = tf.where(
tf.equal(tf.reduce_max(mappa, axis=3, keep_dims=True), mappa), 
tf.constant(1, shape=mappa.shape), 
tf.constant(0, shape=mappa.shape)
)

#Initilizer
init = tf.global_variables_initializer()

#Parameter saving function
saver = tf.train.Saver()  


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Please use `layer.__call__` method instead.
Tensor("maxpool_1:0", shape=(1, 32, 32, 64), dtype=float32)
Tensor("maxpool2:0", shape=(1, 16, 16, 128), dtype=float32)
Tensor("maxpool3:0", shape=(1, 8, 8, 256), dtype=float32)
Tensor("maxpool4:0", shape=(1, 4, 4, 512), dtype=float32)
Tensor("maxpool5:0", shape=(1, 2, 2, 512), dtype=float32)
Instructions for updating:
Use `tf.keras.layer

In [0]:
def Save_img(image,destination,name,corr=1):
  #temp = image[0:len_x,0:len_y,:].astype('uint8')
  temp = image.astype('uint8')
  temp = temp * corr
  img = Image.fromarray(temp, 'RGB')
  os.chdir(destination)
  img.save(name)

In [0]:
def RGB_p_NIR(RGB,origin,name):
  os.chdir(origin)
  NIR = np.load(name)[:,:,0]
  RGB_NIR = np.empty(shape=(RGB.shape[0],RGB.shape[1],N_of_band), dtype=np.float32)
  RGB_NIR[:,:,0:3] = RGB
  RGB_NIR[:,:,3] = NIR[0:NIR.shape[0],0:NIR.shape[1]]
  return RGB_NIR

In [0]:
def Pad_All(RGB_NIR_,tare_,half_ps):
  if (RGB_NIR_.shape[0] % half_ps) > 0:
    x_corr = half_ps - (RGB_NIR_.shape[0] % half_ps)
  else:
    x_corr = 0

  if (RGB_NIR_.shape[1] % half_ps) > 0:
    y_corr = half_ps - (RGB_NIR_.shape[1] % half_ps)
  else:
    y_corr = 0

  RGB_NIR_ = np.pad(RGB_NIR_, ((half_ps,half_ps + x_corr),(half_ps,half_ps + y_corr),(0,0)), 'symmetric')
  tare_ = np.pad(tare_, ((half_ps,half_ps+ x_corr),(half_ps,half_ps + y_corr)), 'symmetric')

  return RGB_NIR_,tare_

In [0]:
def Split_Tare(tare):

  available_classes = np.unique(tare)
  map_600 = np.zeros((tare.shape), dtype = np.float32)
  map_700 = np.zeros((tare.shape), dtype = np.float32)
  map_800 = np.zeros((tare.shape), dtype = np.float32)

  if (638 in available_classes):
    map_600 = np.where(tare == 638, 1, map_600)
  if (659 in available_classes):
    map_600 = np.where(tare == 659, 1, map_600)
  if (654 in available_classes):
    map_600 = np.where(tare == 654, 1, map_600)

  if (656 in available_classes):
    map_700 = np.where(tare == 656, 1, map_700)
  if (650 in available_classes):
    map_700 = np.where(tare == 650, 1, map_700)

  if (660 in available_classes):
    map_800 = np.where(tare == 660, 1, map_800)    
  if (690 in available_classes):
    map_800 = np.where(tare == 690, 1, map_800)
  if (780 in available_classes):
    map_800 = np.where(tare == 780, 1, map_800)
  if (770 in available_classes):
    map_800 = np.where(tare == 770, 1, map_800)
  
  tare_splitted = np.zeros((tare.shape[0],tare.shape[1],N_of_classes), dtype = np.float32)
  tare_splitted[:,:,0] = map_600
  tare_splitted[:,:,1] = map_700
  tare_splitted[:,:,2] = map_800

  tare_gt = np.zeros((1,RGB_NIR.shape[0],RGB_NIR.shape[1],N_of_classes), dtype = np.float32)
  tare_gt[0,:,:,:] = tare_splitted

  return tare_splitted,tare_gt

In [0]:
def cut_image(RGB_NIR, tare_splitted,patch_size):
    N_of_band = RGB_NIR.shape[-1]
    N_of_classes = tare_splitted.shape[-1]

    N_of_patch = int(((RGB_NIR.shape[0]/half_ps) - 1) * ((RGB_NIR.shape[1]/half_ps) - 1))

    #memory allocation
    patch_RGB_NIR = np.zeros([N_of_patch,patch_size,patch_size, N_of_band], dtype = np.float32)
    patch_tare = np.zeros([N_of_patch,patch_size,patch_size,N_of_classes], dtype = np.float32)

    x_dim = RGB_NIR.shape[0]
    y_dim = RGB_NIR.shape[1]

    k = 0

    for i in range(0, (x_dim - half_ps), half_ps):
        for j in range(0,  (y_dim - half_ps), half_ps): 
                patch_RGB_NIR[k,:,:,:] = RGB_NIR[i:i+patch_size,j:j+patch_size,:]
                patch_tare[k,:,:,:] = tare_splitted[i:i+patch_size,j:j+patch_size,:]
                k = k + 1
    
    return patch_RGB_NIR, patch_tare

In [0]:

def reconstruct(list_map,shape):
  #shape as x,y,band
  #input as batch,x,y,band
  mappa = np.zeros(shape, dtype = np.float32)

  x_dim = mappa.shape[0]
  y_dim = mappa.shape[1]
  k = 0
  for i in range(0, (x_dim - half_ps), half_ps):
    for j in range(0,  (y_dim - half_ps), half_ps):
      mappa[i+quarter_ps:i+patch_size-quarter_ps,j+quarter_ps:j+patch_size-quarter_ps,:] = list_map[k,quarter_ps:patch_size-quarter_ps,quarter_ps:patch_size-quarter_ps,:]
      k = k+1
  return mappa


In [0]:
def mask_img(tare,mappa):
  
  mask = np.where(tare == 0, 0, 1)
  mappa[0,:,:,0] = np.multiply(mappa[0,:,:,0],mask)
  mappa[0,:,:,1] = np.multiply(mappa[0,:,:,1],mask)
  mappa[0,:,:,2] = np.multiply(mappa[0,:,:,2],mask)
  return mappa


In [0]:
def mask_img_acc(norm_map,tare):
  mask  = np.where(tare == 0, 0, 1)
  norm_map[0,:,:,0] = np.multiply(norm_map[0,:,:,0],mask)
  norm_map[0,:,:,1] = np.multiply(norm_map[0,:,:,1],mask)
  norm_map[0,:,:,2] = np.multiply(norm_map[0,:,:,2],mask)

  mask  = np.where(tare == 0, 1, 0)
  norm_map[0,:,:,2] = np.add(norm_map[0,:,:,2],mask)
  
  ignore, N_of_pix = np.unique(mask,return_counts=True)

  return norm_map, N_of_pix

In [0]:
def accuracy(list_map,list_gt,patch_size):
  normal = 0
  zeros = 0
  for k in range(0,list_map.shape[0]):
    for i in range(0,patch_size):
      for j in range(0, patch_size):
        if list_gt[k,i,j,0]==list_map[k,i,j,0] and list_gt[k,i,j,0]==list_map[k,i,j,0] and list_gt[k,i,j,0]==list_map[k,i,j,0]:
          normal = normal+1

        if list_gt[k,i,j,0]==0. and list_gt[k,i,j,1]==0. and list_gt[k,i,j,2]==0.:
          zeros=zeros+1
  tot_pix = list_map.shape[0] * list_map.shape[1] *list_map.shape[2]
  tot_pix_ok = tot_pix - zeros
  acc = normal/tot_pix_ok
  string = 'Mask = {}%'.format((zeros/tot_pix)*100)
  return acc,string,normal,zeros

In [0]:
def f_score(y_true, y_pred):

  #80 pascolo
  #160 bosco
  #240 altro

  flat_true = np.zeros([y_true.shape[1], y_true.shape[2]])
  flat_pred = np.zeros([y_pred.shape[1], y_pred.shape[2]])

  temp_f = np.where(y_true[0,:,:,0]==1,80,0)
  flat_true = np.add(flat_true,temp_f) 
  temp_f = np.where(y_true[0,:,:,1]==1,160,0)
  flat_true = np.add(flat_true,temp_f) 
  temp_f = np.where(y_true[0,:,:,2]==1,240,0)
  flat_true = np.add(flat_true,temp_f) 

  temp_f = np.where(y_pred[0,:,:,0]==1,80,0)
  flat_pred = np.add(flat_pred,temp_f) 
  temp_f = np.where(y_pred[0,:,:,1]==1,160,0)
  flat_pred = np.add(flat_pred,temp_f) 
  temp_f = np.where(y_pred[0,:,:,2]==1,240,0)
  flat_pred = np.add(flat_pred,temp_f) 

  flat_true = flat_true.flatten()
  flat_pred = flat_pred.flatten()

  index_f = np.argwhere(flat_true==0)
  flat_true = np.delete(flat_true,index_f)

  flat_pred = np.delete(flat_pred,index_f)

  #print(metrics.classification_report(flat_true, flat_pred, digits=4))

  score = np.zeros((5,3), dtype = np.float32)
  #0 f_score macro - f score wieghted - accuracy -> macro conta le classi non presenti come uno quindi sarà molto alta, wheighted è il più corretto
  #1 prec   grazing (band 0) - forest (band 1) - noncultivable (band 2)
  #2 recal  grazing (band 0) - forest (band 1) - noncultivable (band 2)
  #3 f      grazing (band 0) - forest (band 1) - noncultivable (band 2)
  #4 supp   grazing (band 0) - forest (band 1) - noncultivable (band 2)

  score[0,0] = metrics.f1_score(flat_true, flat_pred, labels=[80,160,240], average='macro', zero_division=1)
  score[0,1] = metrics.f1_score(flat_true, flat_pred, labels=[80,160,240], average='weighted', zero_division=1)
  score[0,2] = metrics.accuracy_score(flat_true, flat_pred)

  prec,rec,f,supp = metrics.precision_recall_fscore_support(flat_true, flat_pred,labels=[80,160,240],average = None,zero_division=1)
  
  score[1,:] = prec
  score[2,:] = rec
  score[3,:] = f
  score[4,:] = supp
  

  return score

In [13]:
############################### ~ PARAMETERS ~ ################################
input_images = '/content/drive/My Drive/Seg_net/dataset/raw/'
output_images = '/content/drive/My Drive/Seg_net/dataset/maps/6_complete64/'
model_data = 'Ep50_Bs32_Ps64_gr1e-05_-extended_ps64_bs32_lr1e-5-_CACHESat_Feb_22_11_36_37_2020/my_model-49'



################################# SET UP #######################################
os.chdir(input_images)
raw_list = os.listdir('.')
#raw_list = ['025122w']
#raw_list = ['041123w']
#patch non presenti in old ma appartenenti a zona semiaffidabile di destra basso
'''raw_list = ['061092w', '061091w', '061063w', '061064w', '061023w', '061142w', '061141w', '061062w', '061061w', '061022w', '061154w', '061114w', '061073w', '061074w', '061033w', '061112w', 
            '061111w', '061072w', '061071w', '061032w', '061031w', '082044w', '061124w', '061083w', '061084w', '061043w', '061044w', '044163w', '082042w', '082041w', '061121w', '061082w', 
            '061081w', '061042w', '061041w', '044162w', '044161w', '062094w', '062053w', '062054w', '062013w', '062014w', '045133w', '045134w', '062091w', '062052w', '062011w', '045132w', 
            '045131w', '062063w', '062064w', '062024w', '045143w', '045144w', '045103w', '045104w', '045063w', '062062w', '062061w', '062021w', '045142w', '045141w', '045102w', '045101w', 
            '045062w', '045061w', '062073w', '062033w', '062034w', '045154w', '062071w', '062032w', '062031w', '045152w', '045151w', '062044w', '045163w']'''

os.chdir(output_images)
already_processed = os.listdir('.')

try:
  acc_tot_vect = np.load('acc_tot_vect.npy')
  f_score_vect = np.load('f_score_vect.npy')
except:
  acc_tot_vect = np.zeros((len(raw_list),2),dtype = np.float32)
  f_score_vect = np.zeros((len(raw_list),5,3),dtype = np.float32)

with tf.Session() as sess:
  sess.run(init)
  saver.restore(sess, '/content/drive/My Drive/Seg_net/model_cache/' + model_data)
  number = 0
  for i in range(0, len(raw_list)):
    #Load data

    os.chdir(output_images)  
    try:    
      os.mkdir(raw_list[i])
      print('creo {}'.format(raw_list[i]))
    except:
      print('{} already created'.format(raw_list[i]))


    if not(raw_list[i] in already_processed):
      number = number + 1
      print('processing {} - {}'.format(number,raw_list[i]))
      t = time.time()
      os.chdir(input_images+raw_list[i])
      RGB = np.load(raw_list[i] + "_RGB.npy")

      RGB_NIR = RGB_p_NIR(RGB,input_images+raw_list[i],raw_list[i] + "_NIR.npy")

      tare = np.load("tare.npy")

      RGB_NIR,tare = Pad_All(RGB_NIR,tare,half_ps)

      Save_img(RGB_NIR[half_ps:len_x+half_ps,half_ps:len_y+half_ps,0:3],output_images + raw_list[i],'1_RGB.png')

      tare_splitted, tare_gt = Split_Tare(tare)

      Save_img(tare_splitted[half_ps:len_x+half_ps,half_ps:len_y+half_ps,:],output_images + raw_list[i],'2_tare.png',255)

      list_rn, list_t = cut_image(RGB_NIR,tare_splitted,patch_size)

      list_map = np.zeros((list_t.shape), dtype = np.float32)

      reconstructed_map = np.zeros((1,RGB_NIR.shape[0],RGB_NIR.shape[1],N_of_classes), dtype = np.float32)
      reconstructed_gt = np.zeros((1,RGB_NIR.shape[0],RGB_NIR.shape[1],N_of_classes), dtype = np.float32)

      for j in range(0,list_rn.shape[0]):
        list_map[j,:,:,:] = sess.run(predictions, feed_dict = {x:list_rn[j:j+1,:,:,:]})

      reconstructed_map[0,:,:,:] = reconstruct(list_map,[RGB_NIR.shape[0],RGB_NIR.shape[1],N_of_classes])
      reconstructed_map = reconstructed_map[:,half_ps:len_x+half_ps,half_ps:len_y+half_ps,:]
      tare_gt = tare_gt[:,half_ps:len_x+half_ps,half_ps:len_y+half_ps,:]

      Save_img(reconstructed_map[0,:,:,:],output_images + raw_list[i],'5_mappa_no_filt.png',255)

      norm_map = sess.run(norm, feed_dict = {mappa:reconstructed_map})

      Save_img(norm_map[0,:,:,:],output_images + raw_list[i],'6_mappa_filt.png',255)

      norm_map,N_of_pix = mask_img_acc(norm_map,tare[half_ps:len_x+half_ps,half_ps:len_y+half_ps])
      Save_img(norm_map[0,:,:,:],output_images + raw_list[i],'7_mappa_filt_fake_masked.png',255)
      acc_N_corr = sess.run(accuracy_tot, feed_dict = {mappa:norm_map,gt:tare_gt})
      acc_tot = acc_N_corr/(N_of_pix[0])
      print('Accuracy of {} = {}'.format(raw_list[i],acc_tot))
      acc_tot_vect[i,0] = acc_tot
      acc_tot_vect[i,1] = N_of_pix[0]


      reconstructed_map = mask_img(tare[half_ps:len_x+half_ps,half_ps:len_y+half_ps],reconstructed_map)
      Save_img(reconstructed_map[0,:,:,:],output_images + raw_list[i],'4_mappa_no_filt_masked.png',255)

      norm_map = sess.run(norm, feed_dict = {mappa:reconstructed_map})
      norm_map = mask_img(tare[half_ps:len_x+half_ps,half_ps:len_y+half_ps],norm_map)
      Save_img(norm_map[0,:,:,:],output_images + raw_list[i],'3_mappa_filt_masked' + str(acc_tot) + '.png',255)

      s = f_score(tare_gt, norm_map)

      f_score_vect[i,:,:] = s

      print('Accuracy of {} = {}'.format(raw_list[i],f_score_vect[i,0,2]))
      print('Mean acc = {}'.format( np.mean(acc_tot_vect[0:number,0])))
      print('F_score mean = {}, f_weighted = {}, Pasc = {}, Bosc = {}, Non-c = {}'.format(np.mean(f_score_vect[0:number,0,1]),f_score_vect[i,0,1],f_score_vect[i,3,0],f_score_vect[i,3,1],f_score_vect[i,3,2]))
        
      np.save('../acc_tot_vect.npy',acc_tot_vect)
      np.save('../f_score_vect.npy',f_score_vect)

      print('Duration: {}'.format(time.time()-t))

INFO:tensorflow:Restoring parameters from /content/drive/My Drive/Seg_net/model_cache/Ep50_Bs32_Ps64_gr1e-05_-extended_ps64_bs32_lr1e-5-_CACHESat_Feb_22_11_36_37_2020/my_model-49
025154w already created
025152w already created
025153w already created
025122w already created
024162w already created
025134w already created
025143w already created
025131w already created
025132w already created
025133w already created
025142w already created
025151w already created
025141w already created
025144w already created
026051w already created
025163w already created
026064w already created
026091w already created
026063w already created
026053w already created
025161w already created
026073w already created
025164w already created
026052w already created
026151w already created
026154w already created
026113w already created
026114w already created
026112w already created
042112w already created
042101w already created
042104w already created
042111w already created
042102w already created
04211

In [14]:
print(raw_list)


['025154w', '025152w', '025153w', '025122w', '024162w', '025134w', '025143w', '025131w', '025132w', '025133w', '025142w', '025151w', '025141w', '025144w', '026051w', '025163w', '026064w', '026091w', '026063w', '026053w', '025161w', '026073w', '025164w', '026052w', '026151w', '026154w', '026113w', '026114w', '026112w', '042112w', '042101w', '042104w', '042111w', '042102w', '042113w', '042103w', '042114w', '041032w', '041042w', '041041w', '041071w', '041043w', '041123w', '041111w', '041112w', '041083w', '041122w', '041124w', '041084w', '041081w', '041082w', '041121w', '042024w', '041163w', '042022w', '042021w', '042023w', '042011w', '042014w', '041162w', '042013w', '042012w', '042052w', '042033w', '042053w', '042051w', '042054w', '042031w', '042043w', '042044w', '042061w', '042032w', '042084w', '042072w', '042081w', '042063w', '042083w', '042074w', '042062w', '042073w', '042082w', '042064w', '042091w', '102063w', '042121w', '042124w', '102074w', '042122w', '102062w', '042123w', '102061w'